In [1]:
%load_ext autoreload
%autoreload 2

from dask_image.ndfilters import median_filter, threshold_local, percentile_filter, uniform_filter, gaussian_filter, sobel
from pyfires.PYF_WindowStats import get_mea_std_window, get_local_stats
from dask_image.ndfilters import convolve, maximum_filter
from satpy.modifiers.angles import _get_sun_angles
from pyfires.PYF_Anisotropy import aniso_diff
from scipy.ndimage import convolve as conv
from cython.parallel import prange
from tqdm import tqdm

import dask
dask.config.set(num_workers=8)

import satpy
satpy.config.set({'cache_dir': "D:/sat_data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from datetime import datetime
from satpy import Scene
import dask.array as da
from glob import glob
from pyfires.PYF_detection import stage1_tests, run_basic_night_detection, run_dets

from pyfires.PYF_detection import stage1_tests, run_basic_night_detection, _make_kern
import pyfires.PYF_Consts as PYFc
from pyfires.PYF_basic import *

from pyspectral.rsr_reader import RelativeSpectralResponse
import xarray as xr
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from satpy import Scene
from glob import glob
import os

from satpy.utils import debug_on
#debug_on()

In [2]:
bdict = {'vi1_band': 'vis_06',
         'vi2_band': 'nir_22',
         'mir_band': 'ir_38',
         'lwi_band': 'ir_105'}


indir = 'D:/sat_data/fci/in/'
tod = 'D:/sat_data/fci/out/'
ifiles_l1 = glob(f'{indir}/*FDHSI*--NC4E_*.nc')
scn = Scene(ifiles_l1, reader='fci_l1c_nc')
scn.load(['true_color'])
scn2 = scn.resample(scn.coarsest_area(), resampler='native')
scn2.save_datasets(base_dir=tod)

Don't know how to open the following files: {'D:/sat_data/fci/in\\W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+FCI-1C-RRAD-FDHSI-FD--CHK-TRAIL---NC4E_C_EUMT_20231120125403_IDPFI_OPE_20231120125007_20231120125924_N__C_0078_0041.nc'}


In [3]:
data_dict, scn, scn2, so, so2 = initial_load(ifiles_l1,  # List of files to load
                                    'fci_l1c_nc',   # The reader to use, in this case the AHI HSD reader
                                     bdict, do_load_lsm=False)       # The bands to load

Don't know how to open the following files: {'D:/sat_data/fci/in\\W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+FCI-1C-RRAD-FDHSI-FD--CHK-TRAIL---NC4E_C_EUMT_20231120125403_IDPFI_OPE_20231120125007_20231120125924_N__C_0078_0041.nc'}
Don't know how to open the following files: {'D:/sat_data/fci/in\\W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+FCI-1C-RRAD-FDHSI-FD--CHK-TRAIL---NC4E_C_EUMT_20231120125403_IDPFI_OPE_20231120125007_20231120125924_N__C_0078_0041.nc'}


In [4]:
data_dict['PFP'] = stage1_tests(data_dict['MIR__BT'],
                                data_dict['BTD'],
                                data_dict['VI1_DIFF'],
                                data_dict['SZA'],
                                data_dict['LSM'],
                                ksizes=[5, 7, 9],
                                do_lsm_mask=True)

In [7]:
data_dict['VI1_DIFF_2'].data = da.where(np.isfinite(data_dict['VI1_DIFF_2']), data_dict['VI1_DIFF_2'], 0)
vid = np.array(data_dict['VI1_DIFF_2']).astype(np.float32)

In [8]:
res_n1_n = aniso_diff(vid,
                niter=1,
                kappa=1)
res_n3_n = aniso_diff(vid,
                niter=2,
                kappa=1)
res_n5_n = aniso_diff(vid,
                niter=3,
                kappa=1)

main_n = np.dstack((vid, res_n1_n, res_n3_n, res_n5_n))

In [9]:
wrap_get_aniso_diffs = dask.delayed(get_aniso_diffs)
main_n2 = wrap_get_aniso_diffs(data_dict['VI1_DIFF_2'], [1, 2, 3])

In [10]:
main_std = np.nanstd(main_n, axis=2)

In [11]:
out = (main_std > 0.01).astype(np.uint8)
out = out * (vid > -0.15)

In [12]:
out2 = out * data_dict['PFP']
#save_output(scn, out2, f'out3', tod, 'B07' )

In [17]:
dets_arr = (out2 > 0).astype(np.uint8)
dets_arr[:,:] = 1
wrap_get_mean_std = dask.delayed(get_mea_std_window)
outa = wrap_get_mean_std(data_dict['PFP'].data,
                         data_dict['VI1_RAD'].data.astype(np.float32),  # VIS chan
                         data_dict['mi_ndfi'].data.astype(np.float32),  # NDFI
                         data_dict['LW1__BT'].data.astype(np.float32),  # LW Brightness Temperature
                         data_dict['BTD'].data.astype(np.float32),  # MIR-LW BTD
                         data_dict['MIR__BT'].data.astype(np.float32),  # MIR BT
                         data_dict['VI1_DIFF'].data.astype(np.float32),  # MIR-LWIR-VIS radiance diff
                         data_dict['LSM'].data,  # The land-sea mask
                         data_dict['LATS'].data.astype(np.float32),  # The pixel longitudes
                         255,  # The value denoting land in the LSM. If 255, ignore mask
                         25)

In [18]:
sdt = datetime.utcnow()
outan = outa.compute()
edt = datetime.utcnow()
print((edt - sdt).total_seconds())

161.123898


In [19]:
perc_good = outan[0, :, :]
n_winpix = outan[1, :, :]
n_cloudpix = outan[2, :, :]
n_waterpix = outan[3, :, :]
mean_lw = outan[4, :, :]
std_lw = outan[5, :, :]
mean_nd = outan[6, :, :]
std_nd = outan[7, :, :]
mean_vi = outan[8, :, :]
std_vi = outan[9, :, :]
mean_btd = outan[10, :, :]
std_btd = outan[11, :, :]
mean_mir = outan[12, :, :]
std_mir = outan[13, :, :]
mean_vid = outan[14, :, :]
std_vid = outan[15, :, :]

In [20]:
#save_output(scn, perc_good, f'perc_good', tod, 'B07' )
#save_output(scn, n_winpix, f'n_winpix', tod, 'B07' )
#save_output(scn, n_cloudpix, f'n_cloudpix', tod, 'B07' )
#save_output(scn, n_waterpix, f'n_waterpix', tod, 'B07' )
#save_output(scn, mean_lw, f'mean_lw', tod, 'B07' )
#save_output(scn, std_lw, f'std_lw', tod, 'B07' )
#save_output(scn, mean_nd, f'mean_nd', tod, 'B07' )
#save_output(scn, std_nd, f'std_nd', tod, 'B07' )
#save_output(scn, mean_vi, f'mean_vi', tod, 'B07' )
#save_output(scn, std_vi, f'std_vi', tod, 'B07' )
#save_output(scn, mean_btd, f'mean_btd', tod, 'B07' )
#save_output(scn, std_btd, f'std_btd', tod, 'B07' )
#save_output(scn, mean_mir, f'mean_mir', tod, 'B07' )
#save_output(scn, std_mir, f'std_mir', tod, 'B07' )
#save_output(scn, mean_vid, f'mean_vid', tod, 'B07' )
#save_output(scn, std_vid, f'std_vid', tod, 'B07' )

In [21]:
#save_output(scn, data_dict['VI1_DIFF'], f'VI1_DIFF', tod, 'B07' )
#save_output(scn, data_dict['MIR__BT'], f'MIR__BT', tod, 'B07' )
#save_output(scn, data_dict['LW1__BT'], f'LW1__BT', tod, 'B07' )
#save_output(scn, data_dict['mi_ndfi'], f'mi_ndfi', tod, 'B07' )
#save_output(scn, data_dict['BTD'], f'BTD', tod, 'B07' )
#save_output(scn, data_dict['VI1_RAD'], f'VI1_RAD', tod, 'B07' )

In [22]:
vi1_diff_stdm = (data_dict['VI1_DIFF'] - mean_vid) / std_vid
mir_bt_stdm = (data_dict['MIR__BT'] - mean_mir) / std_mir
lw1_bt_stdm = (data_dict['LW1__BT'] - mean_lw) / std_lw
mi_ndfi_stdm = (data_dict['mi_ndfi'] - mean_nd) / std_nd
btd_stdm = (data_dict['BTD'] - mean_btd) / std_btd
vi1_rad_stdm = (data_dict['VI1_RAD'] - mean_vi) / std_vi

In [23]:
out3 = out2 * (vi1_diff_stdm > vi1_rad_stdm * 1.5)
#save_output(scn, out3, f'out4', tod, 'B07' )

In [24]:
out4 = out3 * (mir_bt_stdm > 1.5)
#save_output(scn, out4, f'out5', tod, 'B07' )

In [25]:
vid2 = xr.where(mean_vid > 0, mean_vid, 0)
vi1_diff_std2m = (data_dict['VI1_DIFF'] - vid2) / std_vid
#save_output(scn, vi1_diff_std2m, f'VI1_DIFF_STD2', tod, 'B07' )

In [26]:
kern = np.ones((3,3))
fir_d_sum = conv(out4, kern)
local_max = maximum_filter(data_dict['VI1_DIFF'].data, (3,3))
out5 = (fir_d_sum == 1) * (data_dict['VI1_DIFF'] == local_max)
main_out = out4 * (fir_d_sum > 1) + out5 * out4

In [27]:
main_out = out4 * (fir_d_sum > 1) + out5 * out4
#save_output(scn, main_out, f'out6', tod, 'B07' )

In [28]:
main_out = main_out * xr.where(data_dict['MIR__BT'] > mean_mir + 2 * std_mir, main_out, 0)
#save_output(scn, main_out, f'out7', tod, 'B07' )

main_out = main_out * xr.where(data_dict['BTD'] > mean_btd + 2.5, main_out, 0)
#save_output(scn, main_out, f'out8', tod, 'B07' )

main_out = main_out * xr.where(data_dict['BTD'] > mean_btd + 2 * std_btd, main_out, 0)
#save_output(scn, main_out, f'out9', tod, 'B07' )

In [29]:
fir_d_sum = conv(main_out, kern)
local_max = maximum_filter(data_dict['MIR__BT'].data, (3,3))
tmp_out = (fir_d_sum == 1) * (data_dict['MIR__BT'] == local_max)
main_out = main_out * (fir_d_sum > 1) + main_out * tmp_out
#print(np.sum(main_out.data.compute()))
#save_output(scn, main_out, f'out10', tod, 'B07' )

In [30]:
kern = np.array([[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]]) / 5

resx = np.abs(convolve(data_dict['MIR__BT'].data, kern))

kern = kern.T

resy = np.abs(convolve(data_dict['MIR__BT'].data, kern))

res = np.sqrt(resx*resx + resy*resy)

main_out = main_out * (res < 500)

In [31]:
#save_output(scn, resx, f'grad_x', tod, 'B07' )
#save_output(scn, resy, f'grad_y', tod, 'B07' )
#save_output(scn, res, f'grad_res', tod, 'B07' )

In [32]:
#save_output(scn, main_out, f'out11', tod, 'B07' )

In [33]:
mainer = np.array(main_out)
mirbt = np.array(data_dict['MIR__BT'])
viddr = np.array(data_dict['VI1_DIFF'])
btddr = np.array(data_dict['BTD'])

In [34]:
mirdif = np.zeros_like(mirbt)
viddif = np.zeros_like(viddr)
btddif = np.zeros_like(btddr)

for x in tqdm(range(1, viddr.shape[0]-1)):
    for y in range(1, viddr.shape[1]-1):
        
        if mainer[x, y] == 0: continue
            
        n_good = 0
            
        if mainer[x-1, y+1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x-1, y+1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x-1, y+1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x-1, y+1])
            n_good += 1
        if mainer[x, y+1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x, y+1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x, y+1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x, y+1])
            n_good += 1
        if mainer[x+1, y+1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x+1, y+1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x+1, y+1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x+1, y+1])
            n_good += 1
            
        if mainer[x-1, y] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x-1, y])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x-1, y])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x-1, y])
        if mainer[x+1, y] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x+1, y])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x+1, y])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x+1, y])
            n_good += 1
            
        if mainer[x-1, y-1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x-1, y-1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x-1, y-1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x-1, y-1])
            n_good += 1
        if mainer[x, y-1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x, y-1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x, y-1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x, y-1])
            n_good += 1
        if mainer[x+1, y-1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x+1, y-1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x+1, y-1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x+1, y-1])
            n_good += 1
            
        # Account for pixels that may be in the centre of a group of PFPs and hence have no valid neighbours
        if n_good < 1:
            mirdif[x, y] = 1
            viddif[x, y] = 1
            btddif[x, y] = 1
        else:
            mirdif[x, y] = mirdif[x, y] / n_good
            viddif[x, y] = viddif[x, y] / n_good
            btddif[x, y] = btddif[x, y] / n_good
            

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5566/5566 [00:28<00:00, 197.30it/s]


In [39]:
locarr = get_local_stats(mainer, mirbt.astype(np.float32), btddr.astype(np.float32), viddr.astype(np.float32))
mirdif2 = locarr[:, :, 0]
btddif2 = locarr[:, :, 1]
viddif2 = locarr[:, :, 2]

In [40]:
main_out = main_out * (btddif > 1) * (viddif > 0.04)
main_out = main_out * (data_dict['BTD'] > mean_btd + std_mir + std_btd)
#save_output(scn, main_out, f'out14', tod, 'B07' )

In [41]:
kern = np.ones((3,3))
fir_d_sum = conv(main_out, kern)
local_max = maximum_filter(data_dict['VI1_DIFF'].data, (3,3))
out5 = (fir_d_sum == 1) * (data_dict['VI1_DIFF'] == local_max)
main_out2 = main_out * (fir_d_sum > 1) + out5 * main_out
#save_output(scn, main_out2, f'main_out2', tod, 'B07' )

In [42]:
kern_ones = np.ones((3,3))
fir_d_sum = conv(main_out2, kern_ones)
local_max = maximum_filter(data_dict['MIR__BT'].data, (3,3))
out5 = (fir_d_sum == 1) * (data_dict['MIR__BT'] == local_max)
main_out3 = main_out2 * (fir_d_sum > 1) + out5 * main_out2
#save_output(scn, main_out3, f'main_out3', tod, 'B07' )

In [43]:
#############
#TODOTODOTODO
# NEED TO ADD CODE TO RE-ADD MISSING 
# FIRE PIXELS IF THEY ARE BRIGHT ENOUGH 
# AND ARE NEXT TO EXISTING FIRE PIXELS
#TODOTODOTODO
#############

# Absolute MIR BT threshold before a pixel is declared 'fire'
mir_abs_thresh = 350
# BTD thresh for adding back missing pixels
min_btd_addback = 2
max_btd_addback = 15

main_out4 = main_out3 + xr.where(data_dict['MIR__BT'] > mir_abs_thresh, 1, 0).astype(np.uint8)
main_out4 = xr.where(main_out4 > 0, 1, 0).astype(np.uint8)

fir_d_sum = conv(main_out4, kern_ones)

# Threshold for adding missing fire pixels, as the algorithm removes some pixels adjacent to existing detections
# We add back using the BTD weighted by the number of fire pixels adjacent to the candidate.
btd_addback_thresh = (9 - fir_d_sum) * (8 / (max_btd_addback - min_btd_addback)) + min_btd_addback + mean_btd + std_btd
btd_addback_thresh = btd_addback_thresh * data_dict['PFP']
 
main_out5 = main_out4 + xr.where(data_dict['BTD'] > btd_addback_thresh, 1, 0).astype(np.uint8)
main_out5 = xr.where(main_out5 > 0, 1, 0).astype(np.uint8) * \
            xr.where(data_dict['PFP'] > 0, 1, 0).astype(np.uint8) * \
            xr.where(fir_d_sum > 0, 1, 0).astype(np.uint8)
#save_output(scn, main_out5, f'main_out5', tod, 'B07' )

In [44]:
data_dict['fire_dets'] = main_out3
data_dict['mean_mir'] = mean_mir
data_dict['mean_btd'] = mean_btd
data_dict['std_btd'] = std_btd

In [45]:
data_dict = calc_frp(data_dict)

In [46]:
save_output(scn, data_dict['frp_est'], f'frp_est', tod, 'ir_38' )

In [47]:
save_output(scn, data_dict['BTD'], f'BTD', tod, 'ir_38' )

In [37]:
save_output(scn, data_dict['VI1_DIFF'], f'VI1_DIFF', tod, 'ir_38' )

In [ ]:
scn = Scene(ifiles_l15, reader='ahi_hsd')
scn2 = Scene(ifiles_l15, reader='ahi_hsd')

scn.load(['B03', 'B07', 'B13'], calibration='radiance')
scn2.load(['B07', 'B13'])

scn = scn.resample(scn.coarsest_area(), resampler='native')
scn2 = scn2.resample(scn.coarsest_area(), resampler='native')
dd = sort_l1(scn['B03'],
             scn['B07'],
             scn['B13'],
             scn2['B07'],
             scn2['B13'],
             bdict
            )

In [ ]:
bob, bob2 = run_dets(dd)

In [ ]:
save_output(scn, bob, f'dets', tod, 'B07' )
save_output(scn, bob2, f'frp', tod, 'B07' )

In [36]:

save_output(scn, data_dict['BTD'], f'BTD', tod, 'B07' )

KeyError: "No dataset matching 'DataQuery(name='B07')' found"

In [ ]:
def get_data(files):
    scn = Scene(files, reader='fci_l1c_nc')
    scn.load(['ir_38'])
    dd = {'b1': scn['ir_38']}
    return dd

In [ ]:
bb = get_data(ifiles_l15)

In [ ]:
bb.data.compute()

In [45]:
from pyfires.PYF_Rad2TbFuncs import compute_aval

compute_aval({'platform': 'MTG-I1', 'inst': 'fci', 'chan': 'ir_38'})

2.0241550702528723e-07